In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, MultiHeadAttention, LayerNormalization, Add

In [2]:
util.startLoop()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Definir el contrato para GBP/USD
contract = Forex('GBPUSD')

# Establecer el rango de tiempo para la consulta de datos históricos
end_time = datetime.now()
start_time = end_time - timedelta(days=30)

# Obtener los datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='30 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=False
)

# Convertir los datos a DataFrame
df = util.df(bars)

In [5]:
# Calcular indicadores técnicos
df['SMA_20'] = df['close'].rolling(window=20).mean()
df['SMA_50'] = df['close'].rolling(window=50).mean()
df['RSI'] = 100 - (100 / (1 + (df['close'].diff().clip(lower=0).rolling(window=14).mean() /
                                df['close'].diff().clip(upper=0).abs().rolling(window=14).mean())))

macd_fast = df['close'].ewm(span=12, adjust=False).mean()
macd_slow = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = macd_fast - macd_slow

# Eliminar valores NaN
df.dropna(inplace=True)

In [6]:
# Configurar la clave de News API
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=GBP%20USD&from={end_time - timedelta(hours=24)}&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()

In [7]:
# Extraer los titulares relevantes
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")

Titulares de noticias relevantes:
 []


In [8]:
# Contar palabras clave positivas y negativas en las noticias
positive_keywords = ['increase', 'growth', 'positive']
negative_keywords = ['decline', 'decrease', 'negative']

positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))
impact_factor = 1 + (positive_news - negative_news) * 0.001  # Factor de impacto basado en las noticias

In [9]:
# Normalizar datos con MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'SMA_20', 'SMA_50', 'RSI', 'MACD']].values)
scaled_data_adjusted = scaled_data * impact_factor

In [10]:
# Simulación Monte Carlo
num_simulations = 10
monte_carlo_data = []
for _ in range(num_simulations):
    noise = np.random.normal(0, 0.01, scaled_data.shape)
    monte_carlo_data.append(scaled_data + noise)

scaled_data_adjusted = np.vstack([scaled_data_adjusted] + monte_carlo_data)

In [11]:
# KMeans para análisis de patrones
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_assignments = kmeans.fit_predict(scaled_data_adjusted.reshape(-1, 8))  # Ajustar a la cantidad de columnas

# Agregar la información de clusters al modelo
scaled_data_with_clusters = np.column_stack((scaled_data_adjusted.reshape(-1, 8), cluster_assignments.reshape(-1, 1)))

In [12]:
# Preparar datos para la creación de secuencias
sequence_length = 60
num_features = 9  # Ahora incluye el número de columnas ajustado con clusters

x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_with_clusters) - sequence_length):
    x_train.append(scaled_data_with_clusters[i - sequence_length:i])
    y_train.append(scaled_data_with_clusters[i:i + sequence_length])

x_train = np.array(x_train)
y_train = np.array(y_train)

# Crear el modelo Transformer
input_layer = Input(shape=(sequence_length, num_features))

In [13]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = Add()([attention, inputs])
    attention = LayerNormalization(epsilon=1e-6)(attention)

    ff = Dense(ff_dim, activation="relu")(attention)
    ff = Dropout(dropout)(ff)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Add()([ff, attention])
    return LayerNormalization(epsilon=1e-6)(ff)

# Crear arquitectura de atención Transformer
x = transformer_encoder(input_layer, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)

x = Dense(64, activation="relu")(x)
x = Dropout(0.1)(x)
output_layer = Dense(num_features)(x)

# Crear el modelo
model = Model(inputs=input_layer, outputs=output_layer)

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Entrenar el modelo
model.fit(x_train, y_train, epochs=20, batch_size=32)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 9)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 9)     │      9,993 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 60, 9)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 9)     │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 9)     │         18 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 60, 128)   │      1,280 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 128)   │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 60, 9)     │      1,161 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 9)     │          0 │ dense_1[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 9)     │         18 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 9)     │      9,993 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 9)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 9)     │          0 │ dropout_4[0][0],  │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 9)     │         18 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 60, 128)   │      1,280 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 60, 128)   │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 60, 9)     │      1,161 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 9)     │          0 │ dense_3[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 9)     │         18 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 26,165 (102.21 KB)

 Trainable params: 26,165 (102.21 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 0.2892
Epoch 2/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.1477
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.1149
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.1067
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0925
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0829
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0764
Epoch 8/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0725
Epoch 9/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0641
Epoch 10/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0609
Epoch 11/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 0.0508
Epoch 12/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0484
Epoch 13/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0443
Epoch 14/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0444
Epoch 15/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 7

In [15]:
# Realizar predicciones
recent_data = scaled_data_with_clusters[-sequence_length:]  # Últimas entradas para las predicciones
predictions = []
for _ in range(16):  # Generar 16 pasos de predicción
    x_input = np.expand_dims(recent_data, axis=0)  # Agregar la dimensión de batch para el modelo
    predicted_price = model.predict(x_input)  # Predecir
    predictions.append(predicted_price[0])  # Agregar la predicción al array

    # Actualizar recent_data con la predicción para el próximo paso de predicción
    recent_data = np.vstack((recent_data[1:], predicted_price[0]))

# Desnormalizar las predicciones
predictions = scaler.inverse_transform(np.array(predictions)[:, :-1])  # Ajustar solo las columnas necesarias para el análisis

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step


ResourceExhaustedError: Graph execution error:

Detected at node functional_1/multi_head_attention_1/transpose_2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\alex_\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 133, in _run_once

  File "c:\Users\alex_\AppDa